# OpenMM MLTSA example

In [ ]:
from openmmtools import testsystems
import numpy as np 
import mdtraj as md 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 
from simtk import openmm, unit
from simtk.openmm import app
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout



In [ ]:
pdb = PDBFile("topology_new.pdb")
modeller = Modeller(pdb.topology, pdb.positions)
# ... Call some modelling functions here ...
modeller.addSolvent(forcefield)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)


In [ ]:
%%time
n_replicas = 10
reportInterval = 1000
for n in range(n_replicas):
    system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)
    integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
    simulation = Simulation(modeller.topology, system, integrator)
    simulation.context.setPositions(modeller.positions)
    simulation.minimizeEnergy()
    simulation.reporters.append(md.reporters.HDF5Reporter('NaCl_R{}.h5'.format(n), reportInterval, coordinates=True, time=True, cell=True, potentialEnergy=True, temperature=True))
    simulation.reporters.append(StateDataReporter(stdout, 100000, step=True, potentialEnergy=True, temperature=True))
    simulation.step(1000000)
    del simulation
    print("Simulation", n, " Finished")